In [1]:
import pandas as pd  
df = pd.read_csv('train.tsv',header=0,delimiter='\t')  
df.info()  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156060 entries, 0 to 156059
Data columns (total 4 columns):
PhraseId      156060 non-null int64
SentenceId    156060 non-null int64
Phrase        156060 non-null object
Sentiment     156060 non-null int64
dtypes: int64(3), object(1)
memory usage: 4.8+ MB


In [2]:
df.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [3]:
df.Sentiment.value_counts()/df.Sentiment.count()  
#每一个种类所占的百分比

2    0.509945
3    0.210989
1    0.174760
4    0.058990
0    0.045316
Name: Sentiment, dtype: float64

In [4]:
df.Sentiment.value_counts()

2    79582
3    32927
1    27273
4     9206
0     7072
Name: Sentiment, dtype: int64

In [5]:
df.Sentiment.count()  

156060

In [7]:
import numpy as np  
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer  
from sklearn.pipeline import Pipeline  
from sklearn.linear_model import LogisticRegression  
text_clf = Pipeline([('vect', CountVectorizer()),  
                     ('tfidf', TfidfTransformer()),  
                     ('clf', LogisticRegression()),  
                      ])  
text_clf = text_clf.fit(X_train,y_train)  
X_test = df.head()['Phrase']  
predicted = text_clf.predict(X_test)  
print(np.mean(predicted == df.head()['Sentiment']))
for phrase, sentiment in zip(X_test, predicted):  
    print('%r => %s' % (phrase, sentiment))  

0.8
'A series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story .' => 2
'A series of escapades demonstrating the adage that what is good for the goose' => 2
'A series' => 2
'A' => 2
'series' => 2


In [8]:
df.head()['Sentiment']

0    1
1    2
2    2
3    2
4    2
Name: Sentiment, dtype: int64

In [9]:
test_df = pd.read_csv('test.tsv',header=0,delimiter='\t')  
test_df.info()  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66292 entries, 0 to 66291
Data columns (total 3 columns):
PhraseId      66292 non-null int64
SentenceId    66292 non-null int64
Phrase        66292 non-null object
dtypes: int64(2), object(1)
memory usage: 1.5+ MB


In [10]:
from numpy import savetxt  
X_test = test_df['Phrase']  
phraseIds = test_df['PhraseId']  
predicted = text_clf.predict(X_test)  
pred = [[index+156061,x] for index,x in enumerate(predicted)]  
savetxt('lr_benchmark.csv',pred,delimiter=',',fmt='%d,%d',header='PhraseId,Sentiment',comments='')